In [1]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import pandas as pd
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [2]:
def surrogate_gen():
    NUM = 3 # Number of points
    DIM = 2 # Dimension of the points
    C = 10 # regularization parameter (hardness of the margin)
    
    
    # 2-D mean of ones
    M1 = np.ones((DIM,))
    # 2-D mean of threes
    M2 = 3 * np.ones((DIM,))
    # 2-D covariance of 0.3
    C1 = np.diag(0.3 * np.ones((DIM,)))
    # 2-D covariance of 0.2
    C2 = np.diag(0.2 * np.ones((DIM,)))

    # generate 50 points from gaussian 1
    x1 = np.random.multivariate_normal(M1, C1, NUM) #generate gaussian
    # labels
    y1 = np.ones((x1.shape[0],))
    # generate 50 points from gaussian 2
    x2 = np.random.multivariate_normal(M2, C2, NUM)
    y2 = -np.ones((x2.shape[0],))
    # join
    X = np.concatenate((x1, x2), axis = 0)
    y = np.concatenate((y1, y2), axis = 0)

    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [31]:
def surrogate_gen():
    np.random.seed(1)
    n_obs = 252
    n_assets = 150
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fitting convex function to given data

In [19]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    NUM = 3
    u_values = np.linspace(-2.0, 2.0, num=NUM).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [40]:
def surrogate_gen():
    
    # Set dimensions
    m = 40  # number of rows for A
    n = 80  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [3]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")
num_instances = 2
for pkg_idx in range(num_instances):
    ips = []
    #pkg_idx = 0 #identifier index
    success_cnt = 0
    fail_cnt = 0
    max_ipm_steps = 0


    max_iter = 100
    num = 10

    for i in tqdm(range(max_iter)):
        Q, q, G, h, A, b, S = surrogate_gen()
        try:
            res = qp(Q, q, G, h, A, b, callback=lambda res: res)
            max_ipm_steps = max(max_ipm_steps, res['iterations'])
            print("status",res['status'])
        except Exception as e:
            fail_cnt += 1
            warnings.warn(f'Optimization failed with error: {str(e)}')
            continue
        else:
            if res['status'] == 'optimal': #np.isnan(res.fun)?
                #ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float)))
                ips.append((Q, q, G, h, A, b, S))
                success_cnt += 1
                
        if len(ips) >= 1000 or success_cnt == num:
            print("success")
            ips = [ip + (max_ipm_steps,) for ip in ips]
            with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
                pickle.dump(ips, file)
                print(f'Package {pkg_idx} saved')
                pkg_idx += 1

            ips = []
        if success_cnt >= num:
            break
    warnings.resetwarnings()

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:00<00:12,  7.94it/s]

True
     pcost       dcost       gap    pres   dres
 0:  1.4798e+01 -1.5991e+02  3e+02  2e-01  5e-15
 1:  7.6719e+00 -1.6396e+01  3e+01  8e-03  3e-15
 2:  5.7384e-01 -2.6820e+00  3e+00  2e-16  2e-15
 3: -3.6693e-01 -7.5638e-01  4e-01  2e-16  7e-16
 4: -5.0271e-01 -5.4401e-01  4e-02  2e-16  4e-16
 5: -5.3710e-01 -5.3761e-01  5e-04  1e-16  3e-16
 6: -5.3749e-01 -5.3749e-01  5e-06  2e-16  6e-16
 7: -5.3749e-01 -5.3749e-01  5e-08  2e-16  4e-16
Optimal solution found.
status optimal
True
     pcost       dcost       gap    pres   dres
 0:  1.3763e+01 -1.2384e+02  2e+02  2e-01  3e-15
 1:  6.1200e+00 -1.1631e+01  2e+01  4e-03  2e-15
 2:  5.2919e-01 -1.8908e+00  2e+00  2e-16  1e-15
 3: -1.7958e-01 -4.9201e-01  3e-01  2e-16  7e-16
 4: -2.4397e-01 -2.7721e-01  3e-02  2e-16  3e-16
 5: -2.7012e-01 -2.7302e-01  3e-03  1e-16  4e-16
 6: -2.7143e-01 -2.7146e-01  3e-05  3e-16  2e-16
 7: -2.7145e-01 -2.7145e-01  3e-07  2e-16  1e-16
 8: -2.7145e-01 -2.7145e-01  3e-09  2e-16  2e-16
Optimal solution found

  9%|▉         | 9/100 [00:00<00:00, 93.54it/s]

True
     pcost       dcost       gap    pres   dres
 0:  8.2459e+00 -2.4702e+02  4e+02  3e-01  6e-15
 1:  1.1617e+01 -2.6809e+01  4e+01  1e-02  2e-15
 2:  7.8920e-01 -4.4091e+00  5e+00  3e-16  4e-15
 3: -6.8131e-01 -1.2780e+00  6e-01  4e-16  2e-15
 4: -8.5967e-01 -9.1184e-01  5e-02  3e-16  1e-15
 5: -8.8911e-01 -8.8982e-01  7e-04  2e-16  7e-16
 6: -8.8956e-01 -8.8957e-01  7e-06  2e-16  9e-16
 7: -8.8957e-01 -8.8957e-01  7e-08  2e-16  9e-16
Optimal solution found.
status optimal
True
     pcost       dcost       gap    pres   dres
 0:  1.1604e+01 -1.9230e+02  3e+02  2e-01  1e-14
 1:  8.5865e+00 -2.3549e+01  4e+01  2e-02  3e-15
 2:  6.6008e-01 -3.5691e+00  4e+00  6e-16  1e-15
 3: -5.6153e-01 -1.0427e+00  5e-01  2e-16  6e-16
 4: -7.4684e-01 -8.0010e-01  5e-02  2e-16  1e-15
 5: -7.8500e-01 -7.8574e-01  7e-04  1e-16  1e-15
 6: -7.8553e-01 -7.8553e-01  7e-06  1e-16  1e-15
 7: -7.8553e-01 -7.8553e-01  7e-08  2e-16  4e-16
Optimal solution found.
status optimal
True
     pcost       dcost     

In [5]:
# Ensure the output directory exists
os.makedirs('output', exist_ok=True)

# Open and load the .pkl.gz file
for i in range(num_instances):
    with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{i}.pkl.gz', 'rb') as f:
        data = pickle.load(f)

    # Now `data` contains the contents of the pickle file
    output_path = f'output/output{i}.txt'
    with open(output_path, 'w') as output_file:
        output_file.write(str(data))
    print("Data has been written to output/output.txt")

Data has been written to output/output.txt
Data has been written to output/output.txt
